# Final Project: Load balancing

Time is discrete. Let $Q_1$ and $Q_2$ denote the total number of jobs in the server $1$ and $2$, respectively.
Every time slot, the dispatcher observes ($Q_1$, $Q_2$) and takes the action $a_i$, $i = 1, 2$, that dispatches a potential new incoming job to server $i$. The cost in every time slot is $Q_1 + Q_2$, independently of the
action.

In every time slot, there is a probability $\lambda$ of having a new job, which will be dispatched to server
ai. If $Q_i > 0$, with probability $\mu_i$ a job from server i will depart. For simplicity, we will assume that
in every time slot, only one event can happen, i.e., either an arrival, or a departure from servers $1$ or
$2$

For example, in the state $(2,1)$, if the dispatcher takes the action $1$, we have that with probability $\lambda$ the next state will be $(3,1)$, with probability $\mu_1 (1,1)$, , with probability $\mu_2 (2,0)$, and with probability 1 − $\mu_1$ − $\mu_2$ − $\lambda$ the next state will be $(2,1)$.

We will assume that there is an upper bound for both $Q_1$ and $Q_2$ equal to 20. If either $Q_1 = 20$ or $Q_2 = 20$, no new incoming job will arrive to the system.

Let us choose $\mu_1=0.2, \mu_2=0.4$ and $\lambda = 0.3$. Throughout we take that the discounting factor is $\gamma = 0.99$

## 1. MDP

### 1.1. Policy Evaluation

Assume the random policy that dispatches every job with probability $0.5$ to either queue $1$ and $2$.

- Write down the Bellman equation that characterizes the value function for this policy.

- Calculate the value function for this policy using Iterative Policy Evaluation. Due to the contraction principle, the initial vector can be arbitrary, so you can take V (Q1, Q2) = 0, for all (Q1, Q2). To stop iterating, you can take as a criterion that the difference between two iterations must be smaller than some small δ.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

lamda = 0.3
mu1 = 0.2
mu2 = 0.4
gamma = 0.99
p1 = 0.5
p2 = 0.5
p_Q1_Q2_a1 = p1 * (1-mu1-mu2-lamda)
p_Q1_Q2_a2 = p2 * (1-mu1-mu2-lamda)

p_Q1p1_Q2_a1 = p1 * lamda
p_Q1_Q2p1_a2 = p2 * lamda

p_Q1m1_Q2_a1 = p1 * mu1
p_Q1m1_Q2_a2 = p2 * mu1

p_Q1_Q2m1_a1 = p1 * mu2
p_Q1_Q2m1_a2 = p2 * mu2


def value1(epsilon):
    Vn = np.zeros((21,21))
    Vn_p1 = np.zeros((21,21))
    first_boucle = True
    count = 0

    while np.max(abs((Vn_p1 - Vn))) > epsilon or first_boucle:
        count +=1
        first_boucle = False
        action = np.random.randint(0,2)
        Vn = Vn_p1.copy()
        for Q1 in range(21):
            for Q2 in range(21):
                rec = - (Q1 + Q2)

                if Q1 == 20:
                    Vn_Q1p1_Q2 = Vn[Q1,Q2]
                else:
                    Vn_Q1p1_Q2 = Vn[Q1+1,Q2]
                
                if Q2 == 20:
                    Vn_Q1_Q2p1 = Vn[Q1,Q2]
                else:
                    Vn_Q1_Q2p1 = Vn[Q1, Q2+1]

                if Q1 == 0:
                    Vn_Q1m1_Q2 = Vn[Q1,Q2]
                else:
                    Vn_Q1m1_Q2 = Vn[Q1-1,Q2]
                
                if Q2 == 0:
                    Vn_Q1_Q2m1 = Vn[Q1,Q2]
                else:
                    Vn_Q1_Q2m1 = Vn[Q1, Q2-1]


                Vn_p1[Q1,Q2] = rec + gamma * (p_Q1_Q2_a1 * Vn[Q1,Q2]
                                    + p_Q1p1_Q2_a1 * Vn_Q1p1_Q2
                                    + p_Q1_Q2p1_a2 * Vn_Q1_Q2p1
                                    + p_Q1m1_Q2_a1 * Vn_Q1m1_Q2
                                    + p_Q1_Q2m1_a1 * Vn_Q1_Q2m1
                                    + (p_Q1_Q2_a2 * Vn[Q1,Q2]
                                    + p_Q1m1_Q2_a2 * Vn_Q1m1_Q2
                                    + p_Q1_Q2m1_a2 * Vn_Q1_Q2m1))
        
    return Vn_p1, count
                    
epsilon = 1e-5
Value_final, count_final = value1(epsilon)
print(count_final)

In [ ]:
#add plot with titles
plt.imshow(Value_final,cmap='hot')
plt.colorbar()
plt.title('Value function')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

## Optimal Control

In this part you are asked to find the optimal policy to dispatch incoming jobs.

- Write down the Bellman equation that characterizes the optimal policy.

>>> 

- Solve numerically the optimality value function by Value Iteration Algorithm.




In [ ]:
lamda = 0.3
mu1 = 0.2
mu2 = 0.4
gamma = 0.99

p_Q1_Q2_a1 = (1-mu1-mu2-lamda)
p_Q1_Q2_a2 = (1-mu1-mu2-lamda)

p_Q1p1_Q2_a1 = lamda
p_Q1_Q2p1_a2 = lamda

p_Q1m1_Q2_a1 = mu1
p_Q1m1_Q2_a2 = mu1

p_Q1_Q2m1_a1 = mu2
p_Q1_Q2m1_a2 = mu2

actiontab = np.zeros((21,21))


def optimal_value(epsilon):
    Vn = np.zeros((21,21))
    Vn_p1 = np.zeros((21,21))
    first_boucle = True
    count = 0

    while np.max(abs(Vn_p1 - Vn)) > epsilon or first_boucle:
        count +=1
        first_boucle = False
        action = np.random.randint(0,2)
        Vn = Vn_p1.copy()
        for Q1 in range(21):
            for Q2 in range(21):
                rec = - (Q1 + Q2)

                if Q1 == 20:
                    if Q2 == 20:
                        Vn_Q1p1_Q2 = Vn[Q1,Q2]
                        Vn_Q1_Q2p1 = 0
                        actiontab[Q1,Q2] = 3
                    else:
                        Vn_Q1_Q2p1 = Vn[Q1, Q2+1]
                        Vn_Q1p1_Q2 = 0
                        actiontab[Q1,Q2] = 2
                else:
                    if Q2 == 20:
                        Vn_Q1_Q2p1 = 0
                        Vn_Q1p1_Q2 = Vn[Q1+1 ,Q2]
                        actiontab[Q1,Q2] = 1
                    else:
                        if Vn[Q1, Q2+1] >= Vn[Q1+1,Q2]:
                            Vn_Q1_Q2p1 = Vn[Q1, Q2+1]
                            Vn_Q1p1_Q2 = 0
                            actiontab[Q1,Q2] = 2
                        else:
                            Vn_Q1_Q2p1 = 0
                            Vn_Q1p1_Q2 = Vn[Q1+1 ,Q2]
                            actiontab[Q1,Q2] = 1

                if Q1 == 0:
                    Vn_Q1m1_Q2 = Vn[Q1,Q2]
                else:
                    Vn_Q1m1_Q2 = Vn[Q1-1,Q2]
                
                if Q2 == 0:
                    Vn_Q1_Q2m1 = Vn[Q1,Q2]
                else:
                    Vn_Q1_Q2m1 = Vn[Q1, Q2-1]


                Vn_p1[Q1,Q2] = rec + gamma * (p_Q1_Q2_a1 * Vn[Q1,Q2]
                                    + p_Q1p1_Q2_a1 * Vn_Q1p1_Q2
                                    + p_Q1_Q2p1_a2 * Vn_Q1_Q2p1
                                    + p_Q1m1_Q2_a1 * Vn_Q1m1_Q2
                                    + p_Q1_Q2m1_a1 * Vn_Q1_Q2m1)
        
    return Vn_p1, actiontab, count
                    
epsilon = 0.01
Value_final_optimal, optimal_politic, count = optimal_value(epsilon)
print(optimal_politic)
print(count)

In [ ]:
plt.imshow(Value_final_optimal,cmap='hot')
plt.colorbar()
plt.title('Optimal value function')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

In [ ]:
# plot for the optimal action and add values on the cmap

plt.imshow(optimal_politic,cmap='gist_rainbow')

# Add values to each cell
for i in range(optimal_politic.shape[0]):
    for j in range(optimal_politic.shape[1]):
        value = int(optimal_politic[i, j])
        plt.annotate(str(value), (j, i), fontsize=12, ha='center', va='center')


plt.colorbar()
plt.title('Optimal action')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

- Represent on the plane the optimal action as a function of the state (Q1, Q2).

In [ ]:
vecteurs = np.zeros((21,21,2))
for i in range(21):
    for j in range(21):
        new_vector = (optimal_politic[i][j] - 1) * np.array([0,1]) - (optimal_politic[i][j] - 2) * np.array([1,0])
        vecteurs[i,j] = new_vector
print(vecteurs)

fig, ax = plt.subplots()
ax.imshow(Value_final_optimal,cmap='hot')
ax.quiver(vecteurs[:,:,1], vecteurs[:,:,0], angles='xy', scale_units = 'xy', scale=1)
plt.title('Superposition of the optimal value function and the optimal action')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

- Compare the performances obtained with the random policy and the optimal one, how can you conclude that the optimal policy performs better ?

In [ ]:
print(np.mean(Value_final_optimal))
print(np.mean(Value_final))
print(np.array((Value_final_optimal - Value_final) > 0))
# On peut remarquer que la difference entre la politique optimal et la politique aléatoire est bien positive. Ainsi la première est surement meilleure que la deuxième.

We observe that $ \mathbb{E}[V_{\text{optimal}}] > \mathbb{E}[V_{\text{random}}] $, therefore we can conclude that the optimal policy performs better than the random policy.

- Carry out a one-step improvement on the random policy, what do you observe ?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

lamda = 0.3
mu1 = 0.2
mu2 = 0.4
gamma = 0.99
p1 = 0.5
p2 = 0.5
p_Q1_Q2_a1 = p1 * (1-mu1-mu2-lamda)
p_Q1_Q2_a2 = p2 * (1-mu1-mu2-lamda)

p_Q1p1_Q2_a1 = p1 * lamda
p_Q1_Q2p1_a2 = p2 * lamda

p_Q1m1_Q2_a1 = p1 * mu1
p_Q1m1_Q2_a2 = p2 * mu1

p_Q1_Q2m1_a1 = p1 * mu2
p_Q1_Q2m1_a2 = p2 * mu2


def one_step_improvement(epsilon):
    Vn = np.zeros((21,21))
    Vn_p1 = np.zeros((21,21))
    Vn_p1_a1 = np.zeros((21,21))
    Vn_p1_a2 = np.zeros((21,21))
    first_boucle = True
    count = 0

    while np.max(abs((Vn_p1 - Vn))) > epsilon or first_boucle:
        count +=1
        first_boucle = False
        action = np.random.randint(0,2)
        Vn = Vn_p1.copy()
        for Q1 in range(21):
            for Q2 in range(21):
                rec = - (Q1 + Q2)

                if Q1 == 20:
                    Vn_Q1p1_Q2 = Vn[Q1,Q2]
                else:
                    Vn_Q1p1_Q2 = Vn[Q1+1,Q2]
                
                if Q2 == 20:
                    Vn_Q1_Q2p1 = Vn[Q1,Q2]
                else:
                    Vn_Q1_Q2p1 = Vn[Q1, Q2+1]

                if Q1 == 0:
                    Vn_Q1m1_Q2 = Vn[Q1,Q2]
                else:
                    Vn_Q1m1_Q2 = Vn[Q1-1,Q2]
                
                if Q2 == 0:
                    Vn_Q1_Q2m1 = Vn[Q1,Q2]
                else:
                    Vn_Q1_Q2m1 = Vn[Q1, Q2-1]


                Vn_p1[Q1,Q2] = rec + gamma * (p_Q1_Q2_a1 * Vn[Q1,Q2]
                                    + p_Q1p1_Q2_a1 * Vn_Q1p1_Q2
                                    + p_Q1_Q2p1_a2 * Vn_Q1_Q2p1
                                    + p_Q1m1_Q2_a1 * Vn_Q1m1_Q2
                                    + p_Q1_Q2m1_a1 * Vn_Q1_Q2m1
                                    + (p_Q1_Q2_a2 * Vn[Q1,Q2]
                                    + p_Q1m1_Q2_a2 * Vn_Q1m1_Q2
                                    + p_Q1_Q2m1_a2 * Vn_Q1_Q2m1))

    Vn = Vn_p1.copy()
    for Q1 in range(21):
        for Q2 in range(21):
            rec = - (Q1 + Q2)

            if Q1 == 20:
                Vn_Q1p1_Q2 = Vn[Q1,Q2]
            else:
                Vn_Q1p1_Q2 = Vn[Q1+1,Q2]
            
            if Q2 == 20:
                Vn_Q1_Q2p1 = Vn[Q1,Q2]
            else:
                Vn_Q1_Q2p1 = Vn[Q1, Q2+1]

            if Q1 == 0:
                Vn_Q1m1_Q2 = Vn[Q1,Q2]
            else:
                Vn_Q1m1_Q2 = Vn[Q1-1,Q2]
            
            if Q2 == 0:
                Vn_Q1_Q2m1 = Vn[Q1,Q2]
            else:
                Vn_Q1_Q2m1 = Vn[Q1, Q2-1]


            Vn_p1_a1[Q1,Q2] = rec + gamma * (p_Q1_Q2_a1 * Vn[Q1,Q2]
                                + p_Q1p1_Q2_a1 * Vn_Q1p1_Q2
                                + p_Q1m1_Q2_a1 * Vn_Q1m1_Q2
                                + p_Q1_Q2m1_a1 * Vn_Q1_Q2m1)
            
            Vn_p1_a2[Q1,Q2] = rec + gamma * (p_Q1_Q2p1_a2 * Vn_Q1_Q2p1
                                + p_Q1_Q2_a2 * Vn[Q1,Q2]
                                + p_Q1m1_Q2_a2 * Vn_Q1m1_Q2
                                + p_Q1_Q2m1_a2 * Vn_Q1_Q2m1)
            
            if Vn_p1_a1[Q1,Q2] >= Vn_p1_a2[Q1,Q2]:
                Vn_p1[Q1,Q2] = Vn_p1_a1[Q1,Q2]
            else:
                Vn_p1[Q1,Q2] = Vn_p1_a2[Q1,Q2]


    return Vn_p1, count
                    
epsilon = 1e-5
Value_final_one_step_imp, count_final_one_step_imp = one_step_improvement(epsilon)
print(count_final_one_step_imp)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Random policy', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_one_step_imp,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('With one step improvement', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_one_step_imp - Value_final,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()
print((Value_final_one_step_imp - Value_final) > 0)

L'algorithme avec un "one-step improvement" produit des résultats supérieurs à celui basé sur l'évaluation d'une politique aléatoire pour plusieurs raisons :

1. **Local Adjustment:** La méthode "one-step improvement" effectue une ajustement local de la politique en fonction de la valeur actuelle des états, tandis que l'évaluation d'une politique aléatoire peut ne pas exploiter pleinement les informations locales disponibles.

2. **Immediate Benefit:** La procédure "one-step improvement" prend en compte les retours attendus immédiats pour chaque action dans chaque état, ce qui peut rapidement conduire à des améliorations locales significatives.

3. **Use of Current Information:** En utilisant la fonction de valeur actuelle, "one-step improvement" prend des décisions plus informées, tenant compte des connaissances actuelles de l'environnement, ce qui peut conduire à des choix d'actions plus optimaux.

En somme, cette approche profite des informations actuelles pour effectuer des ajustements ciblés, ce qui peut rapidement améliorer les performances par rapport à une politique aléatoire. 


## 2. Tabular Model-Free control

### 2.1 Policy Evaluation

Assume the random policy that dispatches every job with probability 0.5 to either queue 1 and 2. For the learning parameter you can use $\alpha_n = \frac{1}{n}$

- Implement TD(0)

In [ ]:
# Implement TD(0) with probability 0.5 to either queue 1 and 2, and take alpha = 1/n

import numpy as np
import math

lamda = 0.3

mu1 = 0.2
mu2 = 0.4

gamma = 0.99

p1 = 0.5

p2 = 0.5
# Simulate the environment

def simulate_env(Q1, Q2, action):
    Q1_minus_possible = True
    Q2_minus_possible = True
    Q1_plus_possible = True
    Q2_plus_possible = True
    one_action_done = False

    if Q1 == 20:
        Q1_plus_possible = False
        if Q2 == 20:
            Q2_plus_possible = False
            action = 2
        else:
            Q2_plus_possible = True
            action = 1
    else:
        Q1_plus_possible = True
        if Q2 == 20:
            Q2_plus_possible = False
            action = 0
        else:
            Q2_plus_possible = True

    if Q1 == 0:
        Q1_minus_possible = False
    else:
        Q1_minus_possible = True
    
    if Q2 == 0:
        Q2_minus_possible = False
    else:
        Q2_minus_possible = True

    if action == 0:
        if np.random.uniform() < lamda and Q1_plus_possible:
            Q1 += 1
            one_action_done = True
    else:
        if np.random.uniform() < lamda and Q2_plus_possible:
            Q2 += 1
            one_action_done = True
    
    if np.random.uniform() < mu1 and Q1_minus_possible and not one_action_done:
        Q1 -= 1
        one_action_done = True
    if np.random.uniform() < mu2 and Q2_minus_possible and not one_action_done:
        Q2 -= 1
    
    return Q1, Q2

# Implement TD(0)

def TD0(alpha, n):
    Vn = np.zeros((21,21))
    Tirage = np.zeros((21,21))
    first_boucle = True
    count = 0
    Q1 = 0
    Q2 = 0

    while count < n:
        count +=1
        for Q1 in range(21):
            for Q2 in range(21):
                Tirage[Q1,Q2] +=1
                alpha = 1/(Tirage[Q1,Q2])
                # alpha = 1/math.sqrt(math.sqrt(Tirage[Q1,Q2]))
                action = np.random.randint(0,2)
                Q1_new, Q2_new = simulate_env(Q1, Q2, action)
                Vn[Q1,Q2] = Vn[Q1,Q2] + alpha * (- (Q1 + Q2) + gamma * Vn[Q1_new,Q2_new] - Vn[Q1,Q2])

                # Q1 = np.random.randint(0,21)
                # Q2 = np.random.randint(0,21)
    
    return Vn

n = 3000 

Value_final_TD0 = TD0(1, n)
print(Value_final_TD0)



In [ ]:
plt.imshow(Value_final_TD0,cmap='hot')
plt.colorbar()
plt.title('Value function for TD(0)')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

- Compare the obtained value function with the results of Section 1.

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Random policy', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()
print((Value_final_TD0 - Value_final) > 0)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final_one_step_imp,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('With one step improvement', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final_one_step_imp,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()
print((Value_final_TD0 - Value_final_one_step_imp) > 0)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final_optimal,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Optimal value function', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final_optimal,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()

print((Value_final_TD0 - Value_final_optimal) > 0)


- Explore other alternatives for $\alpha_n$, for example constant, to see if convergence improves
- TODO: Copy code from above and change the alpha by: a constant, and 1/sqrt(count), comment the results

In [ ]:
# Implement TD(0) with probability 0.5 to either queue 1 and 2, and take alpha = 1/n

import numpy as np
import math

lamda = 0.3

mu1 = 0.2
mu2 = 0.4

gamma = 0.99

p1 = 0.5

p2 = 0.5
# Simulate the environment

def simulate_env(Q1, Q2, action):
    Q1_minus_possible = True
    Q2_minus_possible = True
    Q1_plus_possible = True
    Q2_plus_possible = True
    one_action_done = False

    if Q1 == 20:
        Q1_plus_possible = False
        if Q2 == 20:
            Q2_plus_possible = False
            action = 2
        else:
            Q2_plus_possible = True
            action = 1
    else:
        Q1_plus_possible = True
        if Q2 == 20:
            Q2_plus_possible = False
            action = 0
        else:
            Q2_plus_possible = True

    if Q1 == 0:
        Q1_minus_possible = False
    else:
        Q1_minus_possible = True
    
    if Q2 == 0:
        Q2_minus_possible = False
    else:
        Q2_minus_possible = True

    if action == 0:
        if np.random.uniform() < lamda and Q1_plus_possible:
            Q1 += 1
            one_action_done = True
    else:
        if np.random.uniform() < lamda and Q2_plus_possible:
            Q2 += 1
            one_action_done = True
    
    if np.random.uniform() < mu1 and Q1_minus_possible and not one_action_done:
        Q1 -= 1
        one_action_done = True
    if np.random.uniform() < mu2 and Q2_minus_possible and not one_action_done:
        Q2 -= 1
    
    return Q1, Q2

# Implement TD(0)

def TD0(alpha, n):
    Vn = np.zeros((21,21))
    Tirage = np.zeros((21,21))
    first_boucle = True
    count = 0
    Q1 = 0
    Q2 = 0

    while count < n:
        count +=1
        for Q1 in range(21):
            for Q2 in range(21):
                Tirage[Q1,Q2] +=1
                alpha = 1
                # alpha = 1/math.sqrt(math.sqrt(Tirage[Q1,Q2]))
                action = np.random.randint(0,2)
                Q1_new, Q2_new = simulate_env(Q1, Q2, action)
                Vn[Q1,Q2] = Vn[Q1,Q2] + alpha * (- (Q1 + Q2) + gamma * Vn[Q1_new,Q2_new] - Vn[Q1,Q2])

                # Q1 = np.random.randint(0,21)
                # Q2 = np.random.randint(0,21)
    
    return Vn

n = 3000

Value_final_TD0 = TD0(1, n)
print(Value_final_TD0)



In [ ]:
plt.imshow(Value_final_TD0,cmap='hot')
plt.colorbar()
plt.title('Value function for TD(0) with alpha = 1')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Random policy', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()
print((Value_final_TD0 - Value_final) > 0)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final_one_step_imp,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('With one step improvement', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final_one_step_imp,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()
print((Value_final_TD0 - Value_final_one_step_imp) > 0)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final_optimal,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Optimal value function', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final_optimal,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()

print((Value_final_TD0 - Value_final_optimal) > 0)


-------------

In [ ]:
# Implement TD(0) with probability 0.5 to either queue 1 and 2, and take alpha = 1/n

import numpy as np
import math

lamda = 0.3

mu1 = 0.2
mu2 = 0.4

gamma = 0.99

p1 = 0.5

p2 = 0.5
# Simulate the environment

def simulate_env(Q1, Q2, action):
    Q1_minus_possible = True
    Q2_minus_possible = True
    Q1_plus_possible = True
    Q2_plus_possible = True
    one_action_done = False

    if Q1 == 20:
        Q1_plus_possible = False
        if Q2 == 20:
            Q2_plus_possible = False
            action = 2
        else:
            Q2_plus_possible = True
            action = 1
    else:
        Q1_plus_possible = True
        if Q2 == 20:
            Q2_plus_possible = False
            action = 0
        else:
            Q2_plus_possible = True

    if Q1 == 0:
        Q1_minus_possible = False
    else:
        Q1_minus_possible = True
    
    if Q2 == 0:
        Q2_minus_possible = False
    else:
        Q2_minus_possible = True

    if action == 0:
        if np.random.uniform() < lamda and Q1_plus_possible:
            Q1 += 1
            one_action_done = True
    else:
        if np.random.uniform() < lamda and Q2_plus_possible:
            Q2 += 1
            one_action_done = True
    
    if np.random.uniform() < mu1 and Q1_minus_possible and not one_action_done:
        Q1 -= 1
        one_action_done = True
    if np.random.uniform() < mu2 and Q2_minus_possible and not one_action_done:
        Q2 -= 1
    
    return Q1, Q2

# Implement TD(0)

def TD0(alpha, n):
    Vn = np.zeros((21,21))
    Tirage = np.zeros((21,21))
    first_boucle = True
    count = 0
    Q1 = 0
    Q2 = 0

    while count < n:
        count +=1
        for Q1 in range(21):
            for Q2 in range(21):
                Tirage[Q1,Q2] +=1
                # alpha = 1
                alpha = 1/math.sqrt(math.sqrt(Tirage[Q1,Q2]))
                action = np.random.randint(0,2)
                Q1_new, Q2_new = simulate_env(Q1, Q2, action)
                Vn[Q1,Q2] = Vn[Q1,Q2] + alpha * (- (Q1 + Q2) + gamma * Vn[Q1_new,Q2_new] - Vn[Q1,Q2])

                # Q1 = np.random.randint(0,21)
                # Q2 = np.random.randint(0,21)
    
    return Vn

n = 3000

Value_final_TD0 = TD0(1, n)
print(Value_final_TD0)



In [ ]:
plt.imshow(Value_final_TD0,cmap='hot')
plt.colorbar()
plt.title('Value function for TD(0) with alpha = 1/sqrt(sqrt(n))')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Random policy', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()
print((Value_final_TD0 - Value_final) > 0)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final_one_step_imp,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('With one step improvement', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final_one_step_imp,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()
print((Value_final_TD0 - Value_final_one_step_imp) > 0)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final_optimal,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Optimal value function', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final_optimal,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()

print((Value_final_TD0 - Value_final_optimal) > 0)


-----------------

In [ ]:
def racine_m(x,m):
    for i in range(m):
        x = math.sqrt(x)
    return x

In [ ]:
# Implement TD(0) with probability 0.5 to either queue 1 and 2, and take alpha = 1/n

import numpy as np
import math

lamda = 0.3

mu1 = 0.2
mu2 = 0.4

gamma = 0.99

p1 = 0.5

p2 = 0.5
# Simulate the environment

def simulate_env(Q1, Q2, action):
    Q1_minus_possible = True
    Q2_minus_possible = True
    Q1_plus_possible = True
    Q2_plus_possible = True
    one_action_done = False

    if Q1 == 20:
        Q1_plus_possible = False
        if Q2 == 20:
            Q2_plus_possible = False
            action = 2
        else:
            Q2_plus_possible = True
            action = 1
    else:
        Q1_plus_possible = True
        if Q2 == 20:
            Q2_plus_possible = False
            action = 0
        else:
            Q2_plus_possible = True

    if Q1 == 0:
        Q1_minus_possible = False
    else:
        Q1_minus_possible = True
    
    if Q2 == 0:
        Q2_minus_possible = False
    else:
        Q2_minus_possible = True

    if action == 0:
        if np.random.uniform() < lamda and Q1_plus_possible:
            Q1 += 1
            one_action_done = True
    else:
        if np.random.uniform() < lamda and Q2_plus_possible:
            Q2 += 1
            one_action_done = True
    
    if np.random.uniform() < mu1 and Q1_minus_possible and not one_action_done:
        Q1 -= 1
        one_action_done = True
    if np.random.uniform() < mu2 and Q2_minus_possible and not one_action_done:
        Q2 -= 1
    
    return Q1, Q2

# Implement TD(0)

def TD0(alpha, n):
    Vn = np.zeros((21,21))
    Tirage = np.zeros((21,21))
    first_boucle = True
    count = 0
    Q1 = 0
    Q2 = 0

    while count < n:
        count +=1
        for Q1 in range(21):
            for Q2 in range(21):
                Tirage[Q1,Q2] +=1
                # alpha = 1
                alpha = 1/racine_m(Tirage[Q1,Q2],4)
                # alpha = 1/math.sqrt(math.sqrt(Tirage[Q1,Q2]))
                action = np.random.randint(0,2)
                Q1_new, Q2_new = simulate_env(Q1, Q2, action)
                Vn[Q1,Q2] = Vn[Q1,Q2] + alpha * (- (Q1 + Q2) + gamma * Vn[Q1_new,Q2_new] - Vn[Q1,Q2])

                # Q1 = np.random.randint(0,21)
                # Q2 = np.random.randint(0,21)
    
    return Vn

n = 3000

Value_final_TD0 = TD0(1, n)
print(Value_final_TD0)



In [ ]:
plt.imshow(Value_final_TD0,cmap='hot')
plt.colorbar()
plt.title('Value function for TD(0) with alpha = 1/sqrt(sqrt(n))')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Value_final,cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Random policy', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Value_final_TD0,cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('TD(0)', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')


ax_2=ax[2].imshow(Value_final_TD0 - Value_final,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Difference of them', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.35)
plt.show()
print((Value_final_TD0 - Value_final) > 0)

2.2 Optimal control

In this part yo uare asked to find the optimal policy to dispatch incoming jobs
- Implement Q-learning


In [ ]:
lamda = 0.3

mu1 = 0.2
mu2 = 0.4

gamma = 0.99
epsilon = 0.1


def Q_learning(alpha, n):
    Q_value = np.zeros((21,21,2))
    Tirage = np.zeros((21,21))
    count = 0
    Q1 = 0
    Q2 = 0

    while count < n:
        count +=1
        Tirage[Q1,Q2] +=1
        alpha = 1/Tirage[Q1,Q2]
        #Deciding the optimal action using epsilon-greedy
        if np.random.rand() < epsilon:
            action = np.random.randint(0,2)
        else:
            action = np.argmax(Q_value[Q1,Q2])
        
        #Simulating a new state
        Q1_new, Q2_new = simulate_env(Q1, Q2, action)
        #Measuring reward
        reward = - (Q1 + Q2)
        Q_value[Q1,Q2,action] = Q_value[Q1,Q2, action] + alpha * ( reward + gamma * np.max(Q_value[Q1_new,Q2_new]) - Q_value[Q1,Q2, action])

        Q1 = Q1_new
        Q2 = Q2_new

    return Q_value

n = 1000000

Final_Q_Value = Q_learning(1, n)
print(Final_Q_Value)

In [ ]:
def action_optimal(Q_value):
    n = len(Q_value)
    action_matrix = np.zeros((n,n))
    max_q_value_matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            action_matrix[i,j] = np.argmax(Q_value[i,j])
            max_q_value_matrix[i,j] = np.max(Q_value[i,j])

    return action_matrix, max_q_value_matrix

In [ ]:
action_matrix, max_Q_value = action_optimal(Q_value = Final_Q_Value)

In [ ]:
fig, ax = plt.subplots(1,2)
ax_0=ax[0].imshow(Final_Q_Value[:,:,0],cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Q value for action 0', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Final_Q_Value[:,:,1],cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('Q value for action 1', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)

plt.show()

In [ ]:
plt.imshow(max_Q_value,cmap = 'hot')
plt.colorbar()
plt.title('Q value for optimal action')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Final_Q_Value[:,:,0],cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Q value for action 0', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Final_Q_Value[:,:,1],cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('Q value for action 1', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')

ax_2=ax[2].imshow(max_Q_value,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Q value for optimal action', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)

plt.show()

- Represent on the plane the optimal action as a function of the state ($Q_1$,$Q_2$)

In [ ]:
vecteurs = np.zeros((21,21,2))
for i in range(21):
    for j in range(21):
        if action_matrix[i,j] == 0:
            vecteurs[i,j] = [1,0]
        else:
            vecteurs[i,j] = [0,1]

fig, ax = plt.subplots()
ax.imshow(max_Q_value,cmap='hot')
ax.quiver(vecteurs[:,:,1], vecteurs[:,:,0], angles='xy', scale_units = 'xy', scale=1)
plt.title('Optimal action')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

- Explore other alternatives for $\alpha_n$, for example constant or $\frac{1}{n}$?, to see if convergence improves

In [ ]:
lamda = 0.3

mu1 = 0.2
mu2 = 0.4

gamma = 0.99
epsilon = 0.1


def Q_learning(alpha, n):
    Q_value = np.zeros((21,21,2))
    Tirage = np.zeros((21,21))
    count = 0
    Q1 = 0
    Q2 = 0

    while count < n:
        count +=1
        Tirage[Q1,Q2] +=1
        # alpha = 1/Tirage[Q1,Q2]
        alpha = 1/math.pow(Tirage[Q1,Q2],1.5)
        #Deciding the optimal action using epsilon-greedy
        if np.random.rand() < epsilon:
            action = np.random.randint(0,2)
        else:
            action = np.argmax(Q_value[Q1,Q2])
        
        #Simulating a new state
        Q1_new, Q2_new = simulate_env(Q1, Q2, action)
        #Measuring reward
        reward = - (Q1 + Q2)
        Q_value[Q1,Q2,action] = Q_value[Q1,Q2, action] + alpha * ( reward + gamma * np.max(Q_value[Q1_new,Q2_new]) - Q_value[Q1,Q2, action])

        Q1 = Q1_new
        Q2 = Q2_new

    return Q_value

n = 1000000

Final_Q_Value = Q_learning(1, n)
action_matrix, max_Q_value = action_optimal(Q_value = Final_Q_Value)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Final_Q_Value[:,:,0],cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Q value for action 0', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Final_Q_Value[:,:,1],cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('Q value for action 1', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')

ax_2=ax[2].imshow(max_Q_value,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Q value for optimal action', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)

plt.show()

In [ ]:
vecteurs = np.zeros((21,21,2))
for i in range(21):
    for j in range(21):
        if action_matrix[i,j] == 0:
            vecteurs[i,j] = [1,0]
        else:
            vecteurs[i,j] = [0,1]

fig, ax = plt.subplots()
ax.imshow(max_Q_value,cmap='hot')
ax.quiver(vecteurs[:,:,1], vecteurs[:,:,0], angles='xy', scale_units = 'xy', scale=1)
plt.title('Optimal action')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()

In [ ]:
lamda = 0.3

mu1 = 0.2
mu2 = 0.4

gamma = 0.99
epsilon = 0.1


def Q_learning(alpha, n):
    Q_value = np.zeros((21,21,2))
    Tirage = np.zeros((21,21))
    count = 0
    Q1 = 0
    Q2 = 0

    while count < n:
        count +=1
        Tirage[Q1,Q2] +=1
        alpha = 1
        # alpha = 1/math.sqrt(math.sqrt(Tirage[Q1,Q2]))
        #Deciding the optimal action using epsilon-greedy
        if np.random.rand() < epsilon:
            action = np.random.randint(0,2)
        else:
            action = np.argmax(Q_value[Q1,Q2])
        
        #Simulating a new state
        Q1_new, Q2_new = simulate_env(Q1, Q2, action)
        #Measuring reward
        reward = - (Q1 + Q2)
        Q_value[Q1,Q2,action] = Q_value[Q1,Q2, action] + alpha * ( reward + gamma * np.max(Q_value[Q1_new,Q2_new]) - Q_value[Q1,Q2, action])

        Q1 = Q1_new
        Q2 = Q2_new

    return Q_value

n = 1000000

Final_Q_Value = Q_learning(1, n)
action_matrix, max_Q_value = action_optimal(Q_value = Final_Q_Value)

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax_0=ax[0].imshow(Final_Q_Value[:,:,0],cmap='hot')
ax[0].set_ylabel('Q2')
ax[0].set_xlabel('Q1')
ax[0].set_title('Q value for action 0', fontsize=14)
sm_0 = plt.cm.ScalarMappable(cmap=ax_0.get_cmap(), norm=ax_0.norm)
sm_0.set_array([])
plt.colorbar(sm_0, ax=ax[0], orientation='vertical')

ax_1=ax[1].imshow(Final_Q_Value[:,:,1],cmap='hot')
ax[1].set_ylabel('Q2')
ax[1].set_xlabel('Q1')
ax[1].set_title('Q value for action 1', fontsize=14)
sm_1 = plt.cm.ScalarMappable(cmap=ax_1.get_cmap(), norm=ax_1.norm)
sm_1.set_array([])
plt.colorbar(sm_1, ax=ax[1], orientation='vertical')

ax_2=ax[2].imshow(max_Q_value,cmap='hot')
ax[2].set_ylabel('Q2')
ax[2].set_xlabel('Q1')
ax[2].set_title('Q value for optimal action', fontsize=14)
sm_2 = plt.cm.ScalarMappable(cmap=ax_2.get_cmap(), norm=ax_2.norm)
sm_2.set_array([])
plt.colorbar(sm_2, ax=ax[2], orientation='vertical')

# Adjust the spacing between subplots
plt.subplots_adjust(wspace=0.3)

plt.show()

In [ ]:
vecteurs = np.zeros((21,21,2))
for i in range(21):
    for j in range(21):
        if action_matrix[i,j] == 0:
            vecteurs[i,j] = [1,0]
        else:
            vecteurs[i,j] = [0,1]

fig, ax = plt.subplots()
ax.imshow(max_Q_value,cmap='hot')
ax.quiver(vecteurs[:,:,1], vecteurs[:,:,0], angles='xy', scale_units = 'xy', scale=1)
plt.title('Optimal action')
plt.xlabel('Q1')
plt.ylabel('Q2')
plt.show()